In [1]:
from sklearn import tree
import numpy as np

import os
import json

# DATASET: GTZAN genre collection

    The dataset consists of 1000 audio tracks each 30 seconds long.
    It contains 10 genres, each represented by 100 tracks.
    The tracks are all 22050Hz Mono 16-bit audio files in .wav format.

    http://marsyas.info/downloads/datasets.html

Procesado con Essentia (http://essentia.upf.edu/documentation/)

    Valores promedio (mean) de los descriptores
    [ 
                'lowlevel.spectral_centroid',
                'lowlevel.dissonance',
                'lowlevel.mfcc',
                'sfx.inharmonicity', 
                'rhythm.bpm
    ]
    +género     
    +id
           

### Ejemplo json

In [2]:
cat dataset/descriptores/blues.00001.json

{"lowlevel.dissonance.mean": "0.435817778111", "lowlevel.mfcc_bands.mean": "0.00309278", "sfx.inharmonicity.mean": "0.072731718421", "rhythm.bpm.mean": "67.7719039917", "lowlevel.spectral_centroid.mean": "2191.51098633", "lowlevel.mfcc.mean": "-695.122", "genre": "blues", "id": "blues.00001"}

In [3]:
files_dir = 'dataset/descriptores'

In [4]:
ls dataset/descriptores

blues.00000.json      blues.00076.json      pop.00025.json
blues.00001.json      blues.00077.json      pop.00026.json
blues.00002.json      blues.00078.json      pop.00027.json
blues.00003.json      blues.00079.json      pop.00028.json
blues.00004.json      blues.00080.json      pop.00029.json
blues.00006.json      blues.00081.json      pop.00030.json
blues.00007.json      blues.00082.json      pop.00031.json
blues.00008.json      blues.00083.json      pop.00032.json
blues.00009.json      blues.00084.json      pop.00033.json
blues.00010.json      blues.00085.json      pop.00034.json
blues.00011.json      blues.00086.json      pop.00035.json
blues.00012.json      blues.00087.json      pop.00036.json
blues.00013.json      blues.00088.json      pop.00037.json
blues.00014.json      blues.00089.json      pop.00038.json
blues.00015.json      blues.00090.json      pop.00039.json
blues.00016.json      blues.00091.json      pop.00040.json
blues.00017.json      blues.00092.json      pop.00041.js

In [13]:
codify_genre = { 
    'country': 0,
    'blues': 1,
    'classical': 2,
    'pop': 3
}
undecode_genre = {v: k for k, v in codify_genre.iteritems()}

def load_song_from_json(json_filename):
    data = json.load( open(json_filename,'r') )
    song_id = data.pop('id')
    song_genre = codify_genre[ str(data.pop('genre')) ]
    x = np.array( [float(v) for (k,v) in data.iteritems()], dtype=float)
    return song_id, undecode_genre[song_genre], x

len_vector = 6 #amount of data value of descriptors
arr = np.empty((0,len_vector), float)
target_arr = np.empty((0,1), float)

for subdir, dirs, files in os.walk(files_dir):
    for f in files:
        if os.path.splitext(f)[1]==".json":
            data = json.load( open(files_dir + "/" + f,'r') )
            song_id = data.pop('id')
            song_genre = codify_genre[ str(data.pop('genre')) ]
            target_arr = np.append(target_arr, song_genre)
            x = np.array( [float(v) for (k,v) in data.iteritems()], dtype=float)
            arr = np.vstack((arr, x))

## Entrenamiento

In [14]:
ct = tree.DecisionTreeClassifier()
ct_entrenado = ct.fit( arr, target_arr ) #cargo los datos (entreno)

def genre_predict(ct_entrenado, x_arr):
    return undecode_genre[ ct_entrenado.predict( x_arr )[0] ]

### Pruebas de predicción

In [17]:
test_id, test_genre, test_arr = load_song_from_json( 'dataset/pruebas/blues.00005.json' )


In [18]:
genre_predict(ct_entrenado, test_arr)

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


'blues'

In [19]:
test_id, test_genre, test_arr = load_song_from_json( 'dataset/pruebas/pop.00010.json' )

In [20]:
test_genre

'pop'

In [21]:
genre_predict(ct_entrenado, test_arr)

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


'pop'